In [ ]:
import os
import random
import warnings
import datetime
import pandas as pd
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import (
    accuracy_score, mean_squared_error,
    mean_absolute_error
)
from scipy.stats import pearsonr
from sklearn.linear_model import Ridge
import json

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
# Set random seeds for reproducibility
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

os.makedirs("comparison_plots", exist_ok=True)

LOADING DATA

In [4]:
def load_and_prepare_data():
    """
    Loads, preprocesses, and transforms time series data for model training.

    This function performs the following steps:
    - Loads and cleans the merged dataset from a CSV file.
    - Converts categorical columns ('Axis' and 'Sober_classification') to numerical codes.
    - Extracts temporal features from the timestamp (e.g., sine and cosine of time of day).
    - Applies a centered rolling average to smooth the 'TAC_Reading' signal.
    - Scales input features and target variable using MinMaxScaler.
    - Generates windowed input-output sequences for time series modeling with a fixed delay.
    - Converts the data into PyTorch tensors for training models like RNNs, LSTMs, or Echo State Networks.

    Returns:
        X_tensor (torch.Tensor): Input tensor of shape (num_samples, window_size, num_features),
                                 containing sequential feature data.
        y_tensor (torch.Tensor): Target tensor of shape (num_samples, 1),
                                 containing delayed TAC readings.
        scaler_y (MinMaxScaler): Scaler object used to normalize the target values,
                                 useful for inverse transforming predictions later.
    """

    data = pd.read_csv("merged_data.csv").dropna()
    data['Axis'] = data['Axis'].astype('category').cat.codes
    data['Sober_classification'] = data['Sober_classification'].astype('category').cat.codes

    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data = data.rename(columns={'pid': 'PID', 'Time': 'time'})
    data['time_index'] = data.groupby('PID').cumcount()

    data['minute_of_day'] = data['timestamp'].dt.hour * 60 + data['timestamp'].dt.minute
    data['sin_time'] = np.sin(2 * np.pi * data['minute_of_day'] / 1440)
    data['cos_time'] = np.cos(2 * np.pi * data['minute_of_day'] / 1440)

    data['TAC_Reading'] = data['TAC_Reading'].rolling(window=3, center=True).mean()
    data = data.dropna()

    features = ['Pe_results', 'Comp_results', 'Axis', 'Sober_classification', 'sin_time', 'cos_time']
    X = data[features].values.astype(np.float32)
    y = data['TAC_Reading'].values.astype(np.float32).reshape(-1, 1)

    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y)

    def create_sequences(X, y, window_size=10, delay=5):
        X_seq, y_seq = [], []
        for i in range(len(X) - window_size - delay):
            X_seq.append(X[i:i+window_size])
            y_seq.append(y[i+window_size+delay])
        return np.array(X_seq), np.array(y_seq)

    X_seq, y_seq = create_sequences(X_scaled, y_scaled, window_size=10, delay=5)

    X_tensor = torch.tensor(X_seq, dtype=torch.float32)
    y_tensor = torch.tensor(y_seq, dtype=torch.float32).view(-1, 1)

    return X_tensor, y_tensor, scaler_y



DEFINING THE MODEL

In [5]:
class EchoStateModel(nn.Module):
    """
    Echo State Network (ESN) implementation for time series modeling.

    This model uses a fixed randomly initialized reservoir with a sparse recurrent 
    weight matrix. The internal reservoir dynamics are governed by a spectral radius 
    and leaky rate. The output is a concatenation of summary statistics from the 
    reservoir states over the input sequence.

    Attributes:
        input_dim (int): Number of input features.
        hidden_dim (int): Number of reservoir neurons.
        output_dim (int): Number of output features (not used in current implementation).
        leaky_rate (float): Leaky integration rate for the reservoir update.
        W_in (nn.Parameter): Input-to-reservoir weight matrix.
        W_res (nn.Parameter): Recurrent reservoir weight matrix (non-trainable).
        bias (nn.Parameter): Bias term for reservoir update (non-trainable).
    """

    def __init__(self, input_dim, hidden_dim, output_dim, spectral_radius=0.9, sparsity=0.05, leaky_rate=0.1):
        """
        Initializes the Echo State Network.

        Args:
            input_dim (int): Dimensionality of the input data.
            hidden_dim (int): Number of hidden reservoir units.
            output_dim (int): Output dimension (can be used for downstream mapping).
            spectral_radius (float): Desired spectral radius of the recurrent weight matrix.
            sparsity (float): Fraction of non-zero connections in the recurrent weight matrix.
            leaky_rate (float): Controls the speed of reservoir state updates.
        """
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.leaky_rate = leaky_rate
        
        self.W_in = nn.Parameter(torch.empty(hidden_dim, input_dim))
        self.W_res = nn.Parameter(torch.empty(hidden_dim, hidden_dim), requires_grad=False)
        self.bias = nn.Parameter(torch.zeros(hidden_dim), requires_grad=False)
        
        self._initialize_weights(spectral_radius, sparsity)

    def _initialize_weights(self, spectral_radius, sparsity):
        """
        Initializes the input and recurrent reservoir weights.

        The recurrent weight matrix is scaled to achieve the desired spectral radius,
        and sparsity is applied to create a sparse reservoir.

        Args:
            spectral_radius (float): Desired spectral radius.
            sparsity (float): Fraction of active connections in the recurrent matrix.
        """
        torch.manual_seed(42)
        nn.init.uniform_(self.W_in, -0.5, 0.5)
        
        mask = torch.rand(self.hidden_dim, self.hidden_dim) < sparsity
        weights = torch.randn(self.hidden_dim, self.hidden_dim) * mask
        
        eigvals = torch.linalg.eigvals(weights).abs()
        scale = spectral_radius / eigvals.max().real
        self.W_res.data = weights * scale

    def forward(self, x):
        """
        Performs a forward pass through the Echo State Network.

        For each timestep in the input sequence, updates the reservoir state using
        leaky integration. Then, returns a summary vector formed by concatenating
        the mean, standard deviation, and last state of the reservoir over the sequence.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, sequence_length, input_dim).

        Returns:
            torch.Tensor: Summary representation of shape (batch_size, hidden_dim * 3).
        """
        batch_size, seq_len, _ = x.shape
        h = torch.zeros(batch_size, self.hidden_dim)
        h_list = []
        
        for t in range(seq_len):
            u = x[:, t, :]
            h = (1 - self.leaky_rate) * h + self.leaky_rate * torch.tanh(
                F.linear(u, self.W_in) + F.linear(h, self.W_res) + self.bias)
            h_list.append(h.unsqueeze(1))
        
        h_stack = torch.cat(h_list, dim=1)
        
        h_summary = torch.cat([
            h_stack.mean(dim=1),
            h_stack.std(dim=1),
            h_stack[:, -1, :]
        ], dim=1)
        
        return h_summary


UTILITY FUNCTIONS


In [6]:
def calculate_rmse(y_true, y_pred):
    """
    Calculates Root Mean Squared Error (RMSE) between true and predicted values.

    Args:
        y_true (np.ndarray): Ground truth values.
        y_pred (np.ndarray): Predicted values.

    Returns:
        float: RMSE value.
    """
    return np.sqrt(mean_squared_error(y_true, y_pred))


def calculate_mae(y_true, y_pred):
    """
    Calculates Mean Absolute Error (MAE) between true and predicted values.

    Args:
        y_true (np.ndarray): Ground truth values.
        y_pred (np.ndarray): Predicted values.

    Returns:
        float: MAE value.
    """
    return mean_absolute_error(y_true, y_pred)


def calculate_normalized_rmse(y_true, y_pred):
    """
    Calculates Normalized Root Mean Squared Error (NRMSE).

    Normalization is done using the range of true values.

    Args:
        y_true (np.ndarray): Ground truth values.
        y_pred (np.ndarray): Predicted values.

    Returns:
        float: NRMSE value.
    """
    rmse = calculate_rmse(y_true, y_pred)
    return rmse / (np.max(y_true) - np.min(y_true))


def plot_predictions_with_boundary(y_train, y_test, y_pred, model_name):
    """
    Plots the true vs predicted TAC readings over the last 200 steps, with a sobriety boundary.

    Args:
        y_train (torch.Tensor): Training ground truth values.
        y_test (torch.Tensor): Test ground truth values.
        y_pred (torch.Tensor): Predicted values for the test set.
        model_name (str): Name of the model for labeling and saving.
    """
    y_train_cpu = y_train.detach().cpu().numpy().flatten()
    y_test_cpu = y_test.detach().cpu().numpy().flatten()
    y_pred_cpu = y_pred.detach().cpu().numpy().flatten()

    full_true = np.concatenate([y_train_cpu, y_test_cpu])
    full_pred = np.concatenate([y_train_cpu, y_pred_cpu])
    true_zoomed = full_true[-200:]
    pred_zoomed = full_pred[-200:]

    plt.figure(figsize=(12, 5))
    plt.plot(true_zoomed, label='True TAC', linewidth=2)
    plt.plot(pred_zoomed, label='Predicted TAC', linewidth=2, linestyle='--')
    plt.axhline(0.08, color='red', linestyle='--', label='Sober Boundary')
    plt.title(f'{model_name} - Last 200 Steps Prediction')
    plt.xlabel('Time Step')
    plt.ylabel('TAC Reading')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()

    os.makedirs("comparison_plots", exist_ok=True)
    plt.savefig(f"comparison_plots/{model_name}_predictions.png")
    plt.close()


def plot_model_metrics_bar(metrics_dict, model_name):
    """
    Plots a bar chart of different metrics for a single model.

    Args:
        metrics_dict (dict): Dictionary of metric names and their corresponding values.
        model_name (str): Name of the model (used for title and saved filename).
    """
    metric_names = list(metrics_dict.keys())
    metric_values = [float(metrics_dict[m]) for m in metric_names]

    plt.figure(figsize=(10, 5))
    bars = plt.bar(metric_names, metric_values, color='lightblue', edgecolor='black')
    plt.title(f'{model_name} - Performance Metrics')
    plt.ylabel('Metric Value')
    plt.grid(axis='y', linestyle='--', alpha=0.6)

    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2.0, height * 1.01, f'{height:.4f}',
                 ha='center', va='bottom', fontsize=10, color='black')

    plt.tight_layout()
    os.makedirs("comparison_plots", exist_ok=True)
    plt.savefig(f'comparison_plots/{model_name}_metrics_bar.png')
    plt.close()


TRAINING MODELS

In [ ]:
def train_model_with_validation(model, X_train, y_train, X_test, y_test, model_name, epochs, lr=0.001):
    """
    Trains an Echo State Network model using ridge regression and evaluates its performance.

    This function simulates training across a given number of epochs by recalculating 
    states and fitting a new ridge regression model at each epoch. It logs training and 
    testing loss curves, saves model weights, and evaluates performance using various metrics.

    Args:
        model (nn.Module): Echo State Network model instance.
        X_train (torch.Tensor): Training input sequences.
        y_train (torch.Tensor): Training target values.
        X_test (torch.Tensor): Test input sequences.
        y_test (torch.Tensor): Test target values.
        model_name (str): Name of the model for saving artifacts.
        epochs (int): Number of training epochs.
        lr (float, optional): Learning rate (not used since ridge regression is non-iterative). Default is 0.001.

    Returns:
        tuple:
            y_pred (torch.Tensor): Predicted values for the test set.
            rmse (float): Root Mean Squared Error.
            mae (float): Mean Absolute Error.
            nrmse (float): Normalized RMSE.
            sobriety_accuracy (float): Accuracy of predicting sobriety status (above/below 0.08 TAC).
            pearson_corr (float): Pearson correlation coefficient between predicted and true TAC.
    """


    model_folder = f'model_{model_name}_{epochs}_epochs'
    os.makedirs(model_folder, exist_ok=True)

    train_losses = []
    test_losses = []
    total_losses = []

    for epoch in range(epochs):
        with torch.no_grad():
            train_states = model(X_train)
            test_states = model(X_test)

        ridge = Ridge(alpha=0.01)
        ridge.fit(train_states.detach().numpy(), y_train.detach().numpy())

        y_pred_train = ridge.predict(train_states.detach().numpy())
        y_pred_test = ridge.predict(test_states.detach().numpy())

        train_loss = mean_squared_error(y_train.detach().numpy(), y_pred_train)
        test_loss = mean_squared_error(y_test.detach().numpy(), y_pred_test)
        total_loss = train_loss + test_loss

        train_losses.append(train_loss)
        test_losses.append(test_loss)
        total_losses.append(total_loss)

        if (epoch + 1) % 10 == 0 or epoch == 0:
            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.6f}, Test Loss: {test_loss:.6f}, Total Loss: {total_loss:.6f}")

    # Save loss curve CSV
    loss_df = pd.DataFrame({
        'epoch': list(range(epochs)),
        'train_loss': train_losses,
        'test_loss': test_losses,
        'total_loss': total_losses
    })
    loss_csv_path = f"{model_folder}/learning_curve.csv"
    loss_df.to_csv(loss_csv_path, index=False)
    print(f"Saved learning curve to: {loss_csv_path}")

    # Final model with all data
    ridge = Ridge(alpha=0.01)
    ridge.fit(train_states.detach().numpy(), y_train.detach().numpy())
    y_pred = ridge.predict(test_states.detach().numpy())
    y_pred = torch.tensor(y_pred, dtype=torch.float32)

    torch.save(model.state_dict(), f'{model_folder}/{model_name}.pth')
    np.save(f'{model_folder}/ridge_weights.npy', ridge.coef_)

    y_test_np = y_test.detach().numpy().flatten()
    y_pred_np = y_pred.detach().numpy().flatten()

    rmse = calculate_rmse(y_test_np, y_pred_np)
    mae = calculate_mae(y_test_np, y_pred_np)
    nrmse = calculate_normalized_rmse(y_test_np, y_pred_np)
    pearson_corr, _ = pearsonr(y_test_np, y_pred_np)

    y_true_class = (y_test_np >= 0.08).astype(int)
    y_pred_class = (y_pred_np >= 0.08).astype(int)
    sobriety_accuracy = accuracy_score(y_true_class, y_pred_class)

    pd.DataFrame({
        'True_TAC': y_test_np,
        'Predicted_TAC': y_pred_np
    }).to_csv(f"{model_folder}/predictions_vs_actuals.csv", index=False)

    pd.DataFrame([{
        'RMSE': rmse,
        'MAE': mae,
        'NRMSE': nrmse,
        'Sobriety_Accuracy': sobriety_accuracy,
        'Pearson_Correlation': pearson_corr
    }]).to_csv(f"{model_folder}/metrics.csv", index=False)

    print(f"{model_name} Sobriety Accuracy: {sobriety_accuracy:.4f}")

    return y_pred, rmse, mae, nrmse, sobriety_accuracy, pearson_corr


MAIN EXECUTION


In [8]:
if __name__ == "__main__":
    """
    Main execution pipeline:
    - Loads and preprocesses the time series data
    - Splits data into training and testing sets
    - Initializes and trains the Echo State Network model
    - Evaluates the model and visualizes predictions and performance metrics
    """

    # Load and prepare the data
    X_tensor, y_tensor, scaler_y = load_and_prepare_data()

    # Split into train/test sets
    X_train, X_test, y_train, y_test = train_test_split(
        X_tensor, y_tensor, test_size=0.2, random_state=seed
    )

    # Initialize the Echo State Network model
    input_dim = X_train.shape[2]
    model = EchoStateModel(
        input_dim=input_dim,
        hidden_dim=1500,
        output_dim=1,
        spectral_radius=0.95,
        sparsity=0.05,
        leaky_rate=0.1
    )

    print(f"\nTraining Reservoir with validation for 300 epochs...")
    y_pred, rmse, mae, nrmse, sobriety_acc, pearson_corr = train_model_with_validation(
        model, X_train, y_train, X_test, y_test, model_name="Reservoir", epochs=300
    )

    # Plot predictions against true values with boundary marker
    plot_predictions_with_boundary(y_train, y_test, y_pred, "Reservoir")

    # Metrics summary dictionary
    metrics = {
        'RMSE': rmse,
        'MAE': mae,
        'NRMSE': nrmse,
        'Sobriety Accuracy': sobriety_acc,
        'Pearson Correlation': pearson_corr
    }

    # Plot performance metrics
    plot_model_metrics_bar(metrics, model_name="Reservoir")

    # Print metrics to console
    print("\n--- Reservoir Metrics ---")
    for metric_name, value in metrics.items():
        print(f"{metric_name}: {value:.4f}")



Training Reservoir with validation for 300 epochs...
Epoch 1/300 - Train Loss: 0.004305, Test Loss: 0.014492, Total Loss: 0.018798
Epoch 10/300 - Train Loss: 0.004305, Test Loss: 0.014492, Total Loss: 0.018798
Epoch 20/300 - Train Loss: 0.004305, Test Loss: 0.014492, Total Loss: 0.018798
Epoch 30/300 - Train Loss: 0.004305, Test Loss: 0.014492, Total Loss: 0.018798
Epoch 40/300 - Train Loss: 0.004305, Test Loss: 0.014492, Total Loss: 0.018798
Epoch 50/300 - Train Loss: 0.004305, Test Loss: 0.014492, Total Loss: 0.018798
Epoch 60/300 - Train Loss: 0.004305, Test Loss: 0.014492, Total Loss: 0.018798
Epoch 70/300 - Train Loss: 0.004305, Test Loss: 0.014492, Total Loss: 0.018798
Epoch 80/300 - Train Loss: 0.004305, Test Loss: 0.014492, Total Loss: 0.018798
Epoch 90/300 - Train Loss: 0.004305, Test Loss: 0.014492, Total Loss: 0.018798
Epoch 100/300 - Train Loss: 0.004305, Test Loss: 0.014492, Total Loss: 0.018798
Epoch 110/300 - Train Loss: 0.004305, Test Loss: 0.014492, Total Loss: 0.0187